In [25]:
import numpy as np
import pandas as pd
from ridge_forecast_2 import ridge_gas_refined

In [32]:
df = pd.read_csv('train.csv')
df['date'] = pd.to_datetime(df['date'])

temp_cols = [c for c in df.columns if 'temp' in c]
wind_cols = [c for c in df.columns if 'wind' in c]

df['avg_temp'] = df[temp_cols].mean(axis=1)
df['avg_wind'] = df[wind_cols].mean(axis=1)
df['is_weekend'] = (df['date'].dt.dayofweek >= 5).astype(int)

# data shift to train on "true" weather
horizon = 14 # NOTE: This will be 7 or 14 depending on horizon (or whatever you want)
feature_cols = ['avg_temp', 'avg_wind', 'is_weekend']
for col in feature_cols:
    df[f'{col}_forecast'] = df[col].shift(-horizon)

df_model = df.dropna().reset_index(drop=True)

x_vars = df_model[[f'{c}_forecast' for c in feature_cols]] # Use the shifted "forecast" weather
y_var = df_model[['demand']]
dates = df_model['date']

# NOTE: split_idx also has to be modified according to horizon
split_idx = 3273
x_train = x_vars.iloc[:split_idx]
y_train, y_test = y_var.iloc[:split_idx], df['demand'].iloc[split_idx:]
dates_train = dates.iloc[:split_idx]

In [35]:
y_test_7 = [138.24691736,
       174.55276601,
       218.37350443,
       224.99747983,
       214.94876055,
       169.44099014,
       162.80386413]

y_test_14 = [211.96657271, 201.90001332, 165.32171839, 153.33096611,
       146.99067778, 142.55464468, 139.6165027 , 138.24691736,
       174.55276601, 218.37350443, 224.99747983, 214.94876055,
       169.44099014, 162.80386413]

In [24]:
def calculate_metrics(real, preds2):
    d = len(real)
    from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
    mse = mean_squared_error(real, preds2)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(real, preds2)
    mape = mean_absolute_percentage_error(real, preds2)
    
    print(f"=== Gas Demand Forecasting Error Metrics ({d} days) ===")
    print(f"MSE:  {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE:  {mae:.4f}")
    print(f"MAPE: {mape:.4%}")

    diff_true = np.diff(real)
    diff_pred = np.diff(preds2)
    directional_acc = np.mean(np.sign(diff_true) == np.sign(diff_pred))
    print(f"Directional Accuracy: {directional_acc:.2%}")

In [ ]:
preds_7, _ = ridge_gas_refined(
    x_train, y_train, dates_train, 
    max_horizon=7, 
    window=7,        
    sig_depth=3,     
    temperature=0.10, 
    level=1,         # NOTE: level for signature kernel distance. lower/higher = rougher/finer comparison
    cv_folds=5
)
calculate_metrics(y_test_7, preds_7)

=== Gas Demand Forecasting Error Metrics (7 days) ===
MSE:  173.0031
RMSE: 13.1531
MAE:  9.6768
MAPE: 5.8816%
Directional Accuracy: 83.33%


In [37]:
# before running this I changed the horizon above
preds_14, _ = ridge_gas_refined(
    x_train, y_train, dates_train, 
    max_horizon=14, 
    window=14,        
    sig_depth=3,     
    temperature=0.10, 
    level=1,         
    cv_folds=10
)
calculate_metrics(y_test_14, preds_14)

=== Gas Demand Forecasting Error Metrics (14 days) ===
MSE:  433.7751
RMSE: 20.8273
MAE:  15.0653
MAPE: 8.3030%
Directional Accuracy: 69.23%
